In [26]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_mt.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,59808.0,...,999 Other,6216 Home Health Care Services,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,NaN
1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,59101.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,NaN
2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,59601.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,NaN
3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,59270.0,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,NaN
4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,59101.0,...,999 Other,5613 Employment Services,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,523626,8146707002,4/8/20,885,PPP,RECIPROCITY INDUSTRIES LLC,2718 MONTANA AVE,BILLINGS,MT,59101.0,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,NaN
3140,523627,8283197000,4/8/20,885,PPP,HEBERLE FORD,PO BOX 5,FORSYTH,MT,59327.0,...,999 Other,4411 Automobile Dealers,4411.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,NaN
3141,523628,8286498608,3/24/21,885,PPS,YOUNG WOMENS CHRISTIAN ASSOCIATION OF BILLINGS...,909 Wyoming Ave,Billings,MT,59101.0,...,999 Other,6241 Individual and Family Services,6241.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,NaN
3142,523629,8968548509,3/10/21,885,PPS,C2C VENTURES INC.,10126 US Highway 2 E,Coram,MT,59913.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,4814097007,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968
1,4358347005,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976
2,7436157007,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265
3,4750337203,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240
4,7853207108,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [27]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MT_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,...,999 Other,6216 Home Health Care Services,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,...,999 Other,2362 Nonresidential Building Construction,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,...,999 Other,5613 Employment Services,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14


In [28]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [29]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [30]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4814097007,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,4358347005,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,7436157007,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,4750337203,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,7853207108,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14
...,...,...,...,...,...
3139,8146707002,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,3.011100e+14
3140,8283197000,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,3.008700e+14
3141,8286498608,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,3.011100e+14
3142,8968548509,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,3.002900e+14


In [31]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
200,7468168403,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,4489387006,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1900,1119017304,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1980,3510988507,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2130,5657827001,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2515,1518898608,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2958,4369657010,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
3000,4365937003,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3008,2539967101,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3102,1698977804,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('30')] #NY
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


  0%|          | 0/3144 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

3144


Matching FIPS to DataFrame

In [11]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:22103]

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4814097007,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,[300630002043002]
1,4358347005,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,[301110002003099]
2,7436157007,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,[300490011012008]
3,4750337203,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,[300830703012043]
4,7853207108,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,[301110004023038]
...,...,...,...,...,...
3139,8146707002,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,[301110004023053]
3140,8283197000,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,[300870001002150]
3141,8286498608,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,[301110011002018]
3142,8968548509,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,[300290001021259]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4814097007,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,300630002043002
1,4358347005,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,301110002003099
2,7436157007,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,300490011012008
3,4750337203,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,300830703012043
4,7853207108,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,301110004023038
...,...,...,...,...,...
3139,8146707002,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,301110004023053
3140,8283197000,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,300870001002150
3141,8286498608,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,301110011002018
3142,8968548509,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,300290001021259


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
200,7468168403,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,4489387006,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1900,1119017304,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1980,3510988507,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2130,5657827001,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2515,1518898608,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2958,4369657010,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
3000,4365937003,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3008,2539967101,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3102,1698977804,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,4814097007,300630002043002
1,4358347005,301110002003099
2,7436157007,300490011012008
3,4750337203,300830703012043
4,7853207108,301110004023038
...,...,...
3139,8146707002,301110004023053
3140,8283197000,300870001002150
3141,8286498608,301110011002018
3142,8968548509,300290001021259


Read in base data

In [18]:
#if from geocode file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_mt.csv")
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MT_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
#fips_check.head() 


,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,59808.0,...,999 Other,6216 Home Health Care Services,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,NaN
1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,59101.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,NaN
2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,59601.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,NaN
3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,59270.0,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,NaN
4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,59101.0,...,999 Other,5613 Employment Services,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,523626,8146707002,4/8/20,885,PPP,RECIPROCITY INDUSTRIES LLC,2718 MONTANA AVE,BILLINGS,MT,59101.0,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,NaN
3140,523627,8283197000,4/8/20,885,PPP,HEBERLE FORD,PO BOX 5,FORSYTH,MT,59327.0,...,999 Other,4411 Automobile Dealers,4411.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,NaN
3141,523628,8286498608,3/24/21,885,PPS,YOUNG WOMENS CHRISTIAN ASSOCIATION OF BILLINGS...,909 Wyoming Ave,Billings,MT,59101.0,...,999 Other,6241 Individual and Family Services,6241.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,NaN
3142,523629,8968548509,3/10/21,885,PPS,C2C VENTURES INC.,10126 US Highway 2 E,Coram,MT,59913.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,NaN


Merge with MainFrame

In [19]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,59808.0,...,6216 Home Health Care Services,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,NaN,300630002043002
1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,59101.0,...,"5413 Architectural, Engineering, and Related S...",5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,NaN,301110002003099
2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,59601.0,...,2362 Nonresidential Building Construction,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,NaN,300490011012008
3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,59270.0,...,6221 General Medical and Surgical Hospitals,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,NaN,300830703012043
4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,59101.0,...,5613 Employment Services,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,NaN,301110004023038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,523626,8146707002,4/8/20,885,PPP,RECIPROCITY INDUSTRIES LLC,2718 MONTANA AVE,BILLINGS,MT,59101.0,...,5415 Computer Systems Design and Related Services,5415.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,NaN,301110004023053
3140,523627,8283197000,4/8/20,885,PPP,HEBERLE FORD,PO BOX 5,FORSYTH,MT,59327.0,...,4411 Automobile Dealers,4411.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,NaN,300870001002150
3141,523628,8286498608,3/24/21,885,PPS,YOUNG WOMENS CHRISTIAN ASSOCIATION OF BILLINGS...,909 Wyoming Ave,Billings,MT,59101.0,...,6241 Individual and Family Services,6241.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,NaN,301110011002018
3142,523629,8968548509,3/10/21,885,PPS,C2C VENTURES INC.,10126 US Highway 2 E,Coram,MT,59913.0,...,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,NaN,300290001021259


<b>Fill FIPS Columns - if FIPS file

In [20]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,59808.0,...,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,NaN,300630002043002,300630002043002
1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,59101.0,...,5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,NaN,301110002003099,301110002003099
2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,59601.0,...,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,NaN,300490011012008,300490011012008
3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,59270.0,...,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,NaN,300830703012043,300830703012043
4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,59101.0,...,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,NaN,301110004023038,301110004023038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,523626,8146707002,4/8/20,885,PPP,RECIPROCITY INDUSTRIES LLC,2718 MONTANA AVE,BILLINGS,MT,59101.0,...,5415.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,NaN,301110004023053,301110004023053
3140,523627,8283197000,4/8/20,885,PPP,HEBERLE FORD,PO BOX 5,FORSYTH,MT,59327.0,...,4411.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,NaN,300870001002150,300870001002150
3141,523628,8286498608,3/24/21,885,PPS,YOUNG WOMENS CHRISTIAN ASSOCIATION OF BILLINGS...,909 Wyoming Ave,Billings,MT,59101.0,...,6241.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,NaN,301110011002018,301110011002018
3142,523629,8968548509,3/10/21,885,PPS,C2C VENTURES INC.,10126 US Highway 2 E,Coram,MT,59913.0,...,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,NaN,300290001021259,300290001021259


Drop Extra FIPS Columns

In [21]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,59808.0,...,999 Other,6216 Home Health Care Services,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,300630002043002
1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,59101.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,301110002003099
2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,59601.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,300490011012008
3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,59270.0,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,300830703012043
4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,59101.0,...,999 Other,5613 Employment Services,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,301110004023038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,523626,8146707002,4/8/20,885,PPP,RECIPROCITY INDUSTRIES LLC,2718 MONTANA AVE,BILLINGS,MT,59101.0,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,301110004023053
3140,523627,8283197000,4/8/20,885,PPP,HEBERLE FORD,PO BOX 5,FORSYTH,MT,59327.0,...,999 Other,4411 Automobile Dealers,4411.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,300870001002150
3141,523628,8286498608,3/24/21,885,PPS,YOUNG WOMENS CHRISTIAN ASSOCIATION OF BILLINGS...,909 Wyoming Ave,Billings,MT,59101.0,...,999 Other,6241 Individual and Family Services,6241.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,301110011002018
3142,523629,8968548509,3/10/21,885,PPS,C2C VENTURES INC.,10126 US Highway 2 E,Coram,MT,59913.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,300290001021259


In [22]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
200,520687,7468168403,2/12/21,885,PPS,HEI INC & SUBSIDIARY,303 Cass Ave,Fairview,MT,59221.0,...,999 Other,2131 Support Activities for Mining,2131.0,MONTANA DISTRICT OFFICE,c $1-2 million,50 to 99,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,520758,4489387006,4/3/20,885,PPP,"ACE ELECTRIC, INC.",808 MAIN ST,LAUREL,MT,59044.0,...,999 Other,2382 Building Equipment Contractors,2382.0,MONTANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1900,522387,1119017304,4/28/20,885,PPP,"MQS, INC",US HIGHWAY 933253 US Hwy 93 N,STEVENSVILLE,MT,59870.0,...,332 Fabricated Metal Products,3323 Architectural and Structural Metals Manuf...,3323.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1980,522467,3510988507,2/24/21,885,PPS,BSDI LLC,6600 Highway 12 W,Lolo,MT,59847.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2130,522617,5657827001,4/6/20,885,PPP,MON KOTA INC,16132 Hwy 200 E PO Box 487,FAIRVIEW,MT,59221.0,...,999 Other,4249 Miscellaneous Nondurable Goods Merchant W...,4249.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2515,523002,1518898608,3/13/21,885,PPP,CHEROKEE CAPITAL INC,104 Central Ave,Hobson,MT,59452.0,...,999 Other,"5242 Agencies, Brokerages, and Other Insurance...",5242.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2958,523445,4369657010,4/3/20,885,PPP,BSDI LLC,6600 HIGHWAY 12 W,LOLO,MT,59847.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
3000,523487,4365937003,4/3/20,885,PPP,"HARPER'S BITTERROOT VALLEY TIRE CENTER, INC",4026 US Hwy 93 N,STEVENSVILLE,MT,59870.0,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3008,523495,2539967101,4/10/20,885,PPP,"CAPTAIN'S MARINE, INC.",3201 US HWY 93 S,KALISPELL,MT,59901.0,...,999 Other,4412 Other Motor Vehicle Dealers,4412.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3102,523589,1698977804,5/21/20,885,PPP,WESTERM STATES ELECTRIC INC,741 Highway 212,RED LODGE,MT,59068.0,...,999 Other,2382 Building Equipment Contractors,2382.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


Convert to File

In [23]:
fips_MT = fips_merge_drop
#fips_TN = fips_merge_drop

In [24]:
fips_MT.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MT_fips_scraped.csv") 

Review & Compare

In [25]:
fips_MT = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MT_fips_scraped.csv") 
fips_MT

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,520487,4814097007,4/4/20,885,PPP,"CONSUMER DIRECT HOLDINGS, INC.",100 Consumer Direct Way,MISSOULA,MT,...,999 Other,6216 Home Health Care Services,6216.0,MONTANA DISTRICT OFFICE,e $5-10 million,500 or more,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,1,520488,4358347005,4/3/20,885,PPP,CTA INC,13 N 23RD ST,BILLINGS,MT,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,2,520489,7436157007,4/7/20,885,PPP,RICHARD G ANDERSON,3424 US HWY 12E,HELENA,MT,...,999 Other,2362 Nonresidential Building Construction,2362.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,3,520490,4750337203,4/27/20,885,PPP,SIDNEY HEALTH CENTER,216 14TH AVE SW,SIDNEY,MT,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,4,520491,7853207108,4/14/20,885,PPP,BETTER BUSINESS SYSTEMS INC,175 North 27th Street Suite 800,BILLINGS,MT,...,999 Other,5613 Employment Services,5613.0,MONTANA DISTRICT OFFICE,e $5-10 million,250 to 499,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,3139,523626,8146707002,4/8/20,885,PPP,RECIPROCITY INDUSTRIES LLC,2718 MONTANA AVE,BILLINGS,MT,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,3.011100e+14
3140,3140,523627,8283197000,4/8/20,885,PPP,HEBERLE FORD,PO BOX 5,FORSYTH,MT,...,999 Other,4411 Automobile Dealers,4411.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,3.008700e+14
3141,3141,523628,8286498608,3/24/21,885,PPS,YOUNG WOMENS CHRISTIAN ASSOCIATION OF BILLINGS...,909 Wyoming Ave,Billings,MT,...,999 Other,6241 Individual and Family Services,6241.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,3.011100e+14
3142,3142,523629,8968548509,3/10/21,885,PPS,C2C VENTURES INC.,10126 US Highway 2 E,Coram,MT,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MONTANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,3.002900e+14
